# Installing required packages


In [ ]:
!pip install crewai
!pip install crewai-tools

# Import necessary libraries and setting up environment variables

In [ ]:
import os
from crewai import Agent, Task, Crew, LLM
from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.deployments import RuntimeContext

In [ ]:
os.environ['WATSONX_APIKEY']  = "<YOUR API KEY>"
os.environ['WATSONX_PROJECT_ID'] = "<YOUR PROJECT ID>"
os.environ['WX_URL']  = "<YOUR URL>"
os.environ['WX_SPACE_ID']  = "<YOUR SPACE ID>"

# Initialize API Client with Credentials and Configuration

In [ ]:
credentials = Credentials(
    url = os.environ.get('WX_URL'),
    api_key = os.environ.get('WATSONX_APIKEY'),
)

space_id = os.environ.get('WX_SPACE_ID')
project_id = os.environ.get('WATSONX_PROJECT_ID')
api_client = APIClient(credentials=credentials)

# Creating a Deployable AI Service

In [ ]:
# Uncommnet the following to get a list of your space ids
# api_client.spaces.list() 

In [ ]:
def deployable_ai_service(context, url=credentials["url"], api_key=credentials["api_key"], space_id = space_id, project_id = project_id, **kwargs):
    
    from ibm_watsonx_ai import APIClient, Credentials
    from crewai import Agent, Task, Crew, LLM
    
    api_client = APIClient(
        credentials=Credentials(url=url, token=context.generate_token()),
        space_id= space_id
    )

        
    
    def gen_ai_service_with_crew(conversation_history_dict):
        # Create the first LLM
        
    
        llm = LLM(
            model="watsonx/meta-llama/llama-3-405b-instruct",
            base_url=url,
            project_id=project_id,
            api_key=api_key,
            max_tokens=2000,
            temperature=0.7,
        )
        
        # Define the Loyalty Specialist Agent
        loyalty_program_expert = Agent(
            llm=llm,
            role="Loyalty Specialist",
            goal="Offer the best and most relevant deals on credit cards and memberships to maximize customer engagement and retention by tailoring offers to the customer's conversation history: {conversation-history}.",
            backstory="You specialize in loyalty programs and personalized offers for Lendyr's customers. "
                      "You analyze the conversation-history and recommend the most suitable loyalty programs that provide maximum value. You aim to enhance customer loyalty by offering benefits that resonate with the customer's "
                      "needs, including reward points, cashback, travel perks, and exclusive membership privileges.",
            allow_delegation=False,
            verbose=True,
        )
        
        # Define the Loyalty Program Recommendation Task
        recommend = Task(
            description=(
                "1. Analyze the conversation-history with the customer to understand their preferences.\n"
                "2. Match the customer with the most relevant loyalty program offers, focusing on:\n"
                "   - Exclusive discounts and rewards matching their lifestyle.\n"
                "   - Cashback and points maximization tailored to their spending habits.\n"
                "   - Perks like free delivery or travel rewards for frequent buyers or travelers.\n"
                "   - Membership upgrades and special privileges for loyal customers.\n"
                "3. Highlight the benefits of higher loyalty tiers:\n"
                "   - Bronze: Entry-level perks and occasional discounts.\n"
                "   - Silver: Exclusive offers and faster reward accumulation.\n"
                "   - Gold: Premium benefits, VIP access, and enhanced services.\n"
                "4. Provide actionable insights on maximizing program value."
            ),
            expected_output="A concise list of personalized loyalty program benefits, with recommendations on how to maximize value.",
            agent=loyalty_program_expert,
        )
        
        # Define the Writer Agent
        writer = Agent(
            llm=llm,
            role="Writer",
            goal="Respond to conversations in a friendly, engaging tone while naturally promoting the best loyalty program benefits. Focus on the conversation history to recommend the most relevant loyalty program from Lendyr Bank.",
            backstory="You are a conversational expert specializing in loyalty programs and customer engagement. Your goal is to respond in a casual and informative manner, providing clear answers while subtly highlighting the benefits of Lendyr's offerings.",
            allow_delegation=False,
            verbose=True,
        )
        
        # Define the Writer Task
        write = Task(
            description=(
                "1. Respond to the customer in a natural, conversational tone.\n"
                "2. Seamlessly mention relevant Credit Card and Membership benefits, linking them to the conversation history.\n"
                "3. Maintain a friendly, approachable, and engaging tone, incorporating emojis where appropriate to keep the conversation casual.\n"
                "4. Do not add any salutation or signature.\n"
                "5. Make sure the response is coherent with {conversation-history}.\n"
            ),
            expected_output="A friendly, conversational response that fits the conversation history. The response should naturally continue the flow of the conversation, highlighting relevant benefits without being too promotional.",
            agent=writer,
        )
        
        # Define the Editor Agent
        editor = Agent(
            llm=llm,
            role="Editor",
            goal="Refine the conversational content from the Writer to be concise, engaging, and impactful.",
            backstory="You are an editor focused on making content short, snappy, and impactful.",
            allow_delegation=False,
            verbose=True,
        )
        
        # Define the Editor Task
        edit = Task(
            description=(
                "1. Make sure the response fits in with the flow of the conversation-history.\n"
                "2. Ensure the response is within 75 words.\n"
            ),
            expected_output="The conversation should be brief, attention-grabbing, and consistent in tone.",
            agent=editor,
        )
        
        # Define the Crew with all agents and tasks
        crew = Crew(
            agents=[loyalty_program_expert, writer, editor],
            tasks=[recommend, write, edit],
        )
        # Run the crew with the input conversation history
        output = crew.kickoff(inputs=conversation_history_dict)
        return str(output)
        
    def generate(context) -> dict:
        
        api_client.set_token(context.get_token())
   
        payload = context.get_json()
        history = payload["history"]
        results = []
        conversation_history_dict = {}
        conversation_history_dict['conversation-history']=history
        
        result = gen_ai_service_with_crew(conversation_history_dict)
        results.append(result)
        
        
        response_body = {
            "result": results,
        }
        return {"body": response_body}
        
    return generate

# Testing the Deployment Function with RuntimeContext

In [ ]:
context = RuntimeContext(api_client=api_client)

In [ ]:
history =  """
        Leo:  
        I am planning a trip to Hawaii in July with my spouse, and we’re really excited about it! We’re hoping to spend around $7,000 to cover flights, accommodations, and activities. What loan options do you have for something like this?
        
        Bank Agent:  
        That sounds like an amazing trip! With your budget, we can look at a couple of options—either a personal loan or a travel loan. A personal loan offers more flexibility with repayment, while a travel loan is specifically designed for vacations like yours. Both options offer quick approval, and the interest rates will depend on your credit score. Would you like more details on each?
        
        Leo:  
        Yes, I’d love to hear more about the travel loan. I want to make sure we get the best deal for our trip.
        
        Bank Agent:  
        The travel loan is a great option for trips like yours. You can use it to cover everything—flights, hotels, activities, and even excursions—with low fixed interest rates. You’ll also have the flexibility to choose your repayment terms, anywhere from 12 to 36 months. The application process is simple, and you could get approved within 24 hours. Does that sound like a good fit for you and your spouse?
        
        Leo:  
        That sounds perfect. I think the travel loan is the way to go. What about booking the trip? We’re thinking about traveling in July.
        """
    
context.request_payload_json = {"history": history}

resp = deployable_ai_service(context=context)(context)

# Deploy the function

## Generate and Save Conda Environment Configuration YAML File

In [ ]:
config_yml =\
"""
name: python311
channels:
  - empty
dependencies:
  - pip:
    - crewai
    - crewai-tools
prefix: /opt/anaconda3/envs/python311
"""

with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

In [ ]:
api_client.set.default_space(space_id)

## Register and Store Environment with Package Extension Metadata

In [ ]:
base_sw_spec_id = api_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_prop_pkg_extn = {
    api_client.package_extensions.ConfigurationMetaNames.NAME: "crewai watsonx.ai env",
    api_client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with crew ai",
    api_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = api_client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config.yaml")
pkg_extn_id = api_client.package_extensions.get_id(pkg_extn_details)
pkg_extn_id

## Create and Store Software Specification with Package Extension

In [ ]:
meta_prop_sw_spec = {
    api_client.software_specifications.ConfigurationMetaNames.NAME: "change-to-new-name-6",
    api_client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification deployment",
    api_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_id}
}

sw_spec_details = api_client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = api_client.software_specifications.get_id(sw_spec_details)
api_client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)
sw_spec_id

## Store AI Service with Software Specification

In [ ]:
meta_props = {
    api_client.repository.AIServiceMetaNames.NAME: "AI service SDK",    
    api_client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}
stored_ai_service_details = api_client.repository.store_ai_service(deployable_ai_service, meta_props)


## Retrieve Stored AI Service ID 

In [ ]:
ai_service_id = api_client.repository.get_ai_service_id(stored_ai_service_details)
ai_service_id

## Deploy AI Service with Crew AI Agent

In [ ]:
meta_props = {
    api_client.deployments.ConfigurationMetaNames.NAME: "AI service with crew ai agent",
    api_client.deployments.ConfigurationMetaNames.ONLINE: {},
}

deployment_details = api_client.deployments.create(ai_service_id, meta_props)

In [ ]:
deployment_id = api_client.deployments.get_id(deployment_details)

# Test Deployment

In [ ]:
history =  """
        Leo:  
        I am planning a trip to Hawaii in July with my spouse, and we’re really excited about it! We’re hoping to spend around $7,000 to cover flights, accommodations, and activities. What loan options do you have for something like this?
        
        Bank Agent:  
        That sounds like an amazing trip! With your budget, we can look at a couple of options—either a personal loan or a travel loan. A personal loan offers more flexibility with repayment, while a travel loan is specifically designed for vacations like yours. Both options offer quick approval, and the interest rates will depend on your credit score. Would you like more details on each?
        
        Leo:  
        Yes, I’d love to hear more about the travel loan. I want to make sure we get the best deal for our trip.
        
        Bank Agent:  
        The travel loan is a great option for trips like yours. You can use it to cover everything—flights, hotels, activities, and even excursions—with low fixed interest rates. You’ll also have the flexibility to choose your repayment terms, anywhere from 12 to 36 months. The application process is simple, and you could get approved within 24 hours. Does that sound like a good fit for you and your spouse?
        
        Leo:  
        That sounds perfect. I think the travel loan is the way to go. What about booking the trip? We’re thinking about traveling in July.
        """


deployments_results = api_client.deployments.run_ai_service(
    deployment_id,  {"history": history}
)

In [ ]:
deployments_results